In [1]:
import numpy as np
import zdm.analyze_cube as ac

In [2]:
cube_dir  = "../CRACO/Cubes/craco_mini_cube.npz"
cube_data = np.load(cube_dir)

In [3]:
def get_param_values(data,params):
    """
    Gets the unique values of the data from a cube output
    Currently the parameter order is hard-coded
    
    """
    # gets unique values for each axis
    param_vals=[]
    #param_list=[data["lEmax"],data["H0"],data["alpha"],data["gamma"],data["sfr_n"],data["lmean"],data["lsigma"]]
    #for col in param_list:
    for param in params:
        col=data[param]
        unique=np.unique(col)
        param_vals.append(unique)  
    return param_vals

In [4]:
ll = cube_data["ll"]
params = cube_data["params"]
param_vals = get_param_values(cube_data, params)

In [12]:
uvals=[]
latexnames=[]
for ip,param in enumerate(cube_data["params"]):
    # switches for alpha
    if param=="alpha":
        uvals.append(cube_data[param]*-1.)
    else:
        uvals.append(cube_data[param])
    if param=="alpha":
        latexnames.append('$\\alpha$')
        ialpha=ip
    elif param=="F":
        latexnames.append('$F$')
    elif param=="lEmax":
        latexnames.append('$\\log_{10} E_{\\rm max}$')
    elif param=="H0":
        latexnames.append('$H_0$')
    elif param=="gamma":
        latexnames.append('$\\gamma$')
    elif param=="sfr_n":
        latexnames.append('$n_{\\rm sfr}$')
    elif param=="lmean":
        latexnames.append('$\\mu_{\\rm host}$')
    elif param=="lsigma":
        latexnames.append('$\\sigma_{\\rm host}$')

In [6]:
deprecated, uw_vectors, wvectors=ac.get_bayesian_data(cube_data["ll"])

In [7]:
list2 = []
vals2 = []

for i, vec in enumerate(wvectors):
  n_idx = np.argmax(vec)
  val = uvals[i][n_idx]

  if params[i] != "F":
    list2.append(params[i])
    vals2.append(val)
  else:
    iF = i


In [14]:
for i, p in enumerate(list2):
  list3 = np.concatenate((list2[0:i], list2[i+1:]))
  vals3 = np.concatenate((vals2[0:i], vals2[i+1:]))
  arr = ac.get_slice_from_parameters(cube_data, list3, vals3)

  nans = np.isnan(arr)
  arr[nans] = -9e9

  arr -= np.max(arr)
  arr = 10**arr
  arr /= np.sum(arr)

  if i < iF:
    modi = i
  else:
    modi = i+1
    arr = arr.swapaxes(0,1)

  savename = f"2D Plots/{params[iF]}_{params[modi]}.pdf"

  if params[modi] == "alpha":
    arr = np.flip(arr, axis=0)
    ac.make_2d_plot(arr, latexnames[modi], latexnames[iF], -param_vals[modi], param_vals[iF], savename=savename, norm=1)
  else:
    ac.make_2d_plot(arr, latexnames[modi], latexnames[iF], param_vals[modi], param_vals[iF], savename=savename, norm=1)